<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multimodal Template

In [8]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [9]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [11]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-28 08:05:20,319 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-28 08:05:20,320 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [12]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-C61G" # rid



In [13]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-28 08:05:23,651 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-28 08:05:23,652 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [14]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-28 08:05:24,217 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_join.json] transfer successful. 0.68 KB transferred. Elapsed time: 0:00:00.000103.
2024-06-28 08:05:24,218 - INFO - Verifying SHA256 checksum for downloaded file [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_join.json]
2024-06-28 08:05:24,254 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/sreenidhi/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'multimodal_data_analysis',
    'rid': '2-BDAC'}],
  'Execution_Asset_Type': [{'name': 'Multimodal_Analysis', 'rid': '2-BDAE'}]},
 'execution_rid': '2-C8S6',
 'workflow_rid': '2-BDAG',
 'bag_paths': [PosixPath('/data/2-7P5P_1c2dda3d961d50c2b541eb05ae87ad35ebe02b04402894b42dc5ed594cef944c/Dataset_2-7P5P')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-multimodal_df_join.json')}

In [29]:
# View data

subject = pd.read_csv(configuration_records.bag_paths[0]/'data/Subject.csv')
subject

observation = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation.csv')
observation

clinic = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinical_Records.csv')
clinic

observation_clinic_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation_Clinic_Asso.csv')
observation_clinic_asso # association table between observation table and clinic record table

# icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv')
# icd10

# icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv')
# icd10_asso # association table between clinic record table and ICD10 code

# report = pd.read_csv(configuration_records.bag_paths[0]/'data/Report.csv')
# report

# RNFL_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/RNFL_OCR.csv')
# RNFL_OCR

# HVF_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/HVF_OCR.csv')
# HVF_OCR

print()

In [30]:
subject


,RID,RCT,RMT,RCB,RMB,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,Subject_Ethnicity,Subject_hypertension,Subject_insulin_dependent,Subject_pregnant,Subject_cataract,Subject_maculopathy,Subject_other,Subject_image_quality,site_mrn,Consultant
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2-7KTW,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,3,NaN,NaN,NaN,NaN,...,69WJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2-7KTY,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,4,NaN,NaN,NaN,NaN,...,69WM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2-7KV0,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,5,NaN,NaN,NaN,NaN,...,69WJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,2-7P5C,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1195,NaN,NaN,NaN,NaN,...,69WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,2-7P5E,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1196,NaN,NaN,NaN,NaN,...,69WM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,2-7P5G,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1197,NaN,NaN,NaN,NaN,...,69WR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,69WE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
observation

,RID,RCT,RMT,RCB,RMB,Observation_ID,Subject,Date_of_Encounter
0,2-7RGP,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,2-7KTR,2020-08-05 00:00:00+00
1,2-7RGR,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,2-7KTR,2020-10-22 00:00:00+00
2,2-7RGT,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,659,2-7KTT,2023-04-18 00:00:00+00
3,2-7RGW,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,660,2-7KTT,2023-04-19 00:00:00+00
4,2-7RGY,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,658,2-7KTT,2022-12-20 00:00:00+00
...,...,...,...,...,...,...,...,...
2541,2-7XFG,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,471,2-7P5J,2022-09-14 00:00:00+00
2542,2-7XFJ,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,470,2-7P5J,2022-04-20 00:00:00+00
2543,2-7XFM,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,474,2-7P5M,2023-04-19 00:00:00+00
2544,2-7XFP,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,473,2-7P5M,2022-10-19 00:00:00+00


In [32]:
# Merge Subject with Observation
subject_observation = pd.merge(subject, observation, left_on='RID', right_on='Subject', how='left', suffixes=('_subject_df', '_observation_df'))
subject_observation

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,site_mrn,Consultant,RID_observation_df,RCT_observation_df,RMT_observation_df,RCB_observation_df,RMB_observation_df,Observation_ID,Subject,Date_of_Encounter
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGP,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,2-7KTR,2020-08-05 00:00:00+00
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGR,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,2-7KTR,2020-10-22 00:00:00+00
2,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGT,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,659,2-7KTT,2023-04-18 00:00:00+00
3,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGW,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,660,2-7KTT,2023-04-19 00:00:00+00
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7RGY,2024-04-15 23:50:33.32379+00,2024-04-15 23:50:33.32379+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,658,2-7KTT,2022-12-20 00:00:00+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7XFG,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,471,2-7P5J,2022-09-14 00:00:00+00
2542,2-7P5J,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1198,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7XFJ,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,470,2-7P5J,2022-04-20 00:00:00+00
2543,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7XFM,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,474,2-7P5M,2023-04-19 00:00:00+00
2544,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,NaN,2-7XFP,2024-04-15 23:50:35.895301+00,2024-04-15 23:50:35.895301+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,473,2-7P5M,2022-10-19 00:00:00+0

In [33]:
# Merge with Observation_Clinical_Association
subject_obs_clinic = pd.merge(subject_observation, observation_clinic_asso, left_on= 'RID_observation_df', right_on='Observation', suffixes=('_subject_observation_df', "_observation_clinic_asso_df"), how='left')

subject_obs_clinic

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,Observation_ID,Subject,Date_of_Encounter,RID,RCT,RMT,RCB,RMB,Clinical_Records,Observation
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,1,2-7KTR,2020-08-05 00:00:00+00,2-8QZY,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT4,2-7RGP
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,1,2-7KTR,2020-08-05 00:00:00+00,2-8R00,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT6,2-7RGP
2,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,2,2-7KTR,2020-10-22 00:00:00+00,2-8R02,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XT8,2-7RGR
3,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,2,2-7KTR,2020-10-22 00:00:00+00,2-8R04,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTA,2-7RGR
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,659,2-7KTT,2023-04-18 00:00:00+00,2-8R06,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-7XTC,2-7RGT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,474,2-7P5M,2023-04-19 00:00:00+00,2-91GR,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868W,2-7XFM
4971,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,473,2-7P5M,2022-10-19 00:00:00+00,2-91GT,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-868Y,2-7XFP
4972,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,473,2-7P5M,2022-10-19 00:00:00+00,2-91GW,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8690,2-7XFP
4973,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,472,2-7P5M,2022-04-20 00:00:00+00,2-91GY,2024-04-15 23:57:26.157971+00,2024-04-15 23:57:26.157971+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2-8692,2-7XFR


In [34]:
clinic

,RID,RCT,RMT,RCB,RMB,Date_of_Encounter,LogMAR_VA,Visual_Acuity_Numerator,IOP,Refractive_Error,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label
0,2-7XT4,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-08-05 00:00:00,0.14,20.0,13.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130,2-C60M
1,2-7XT6,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-08-05 00:00:00,0.22,20.0,14.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131,2-C60M
2,2-7XT8,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-10-22 00:00:00,0.00,20.0,14.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132,2-C60M
3,2-7XTA,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-10-22 00:00:00,0.00,20.0,13.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133,2-C60M
4,2-7XTC,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2023-04-18 00:00:00,0.18,20.0,9.0,-0.5,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847,2-C60M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4332,2-868W,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2023-04-19 00:00:00,0.18,20.0,19.0,NaN,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014,2-C60J
4333,2-868Y,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-10-19 00:00:00,0.18,20.0,16.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728,2-C60J
4334,2-8690,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-10-19 00:00:00,0.22,20.0,16.0,NaN,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729,2-C60J
4335,2-8692,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-04-20 00:00:00,0.18,20.0,14.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726,2-C60J


In [35]:
clinic['Condition_Label'].unique()

array(['2-C60M', '2-C60J', nan, '2-C60P'], dtype=object)

In [36]:
import numpy as np

# Create the first new column with condition names
condition_map = {
    '2-C60J': 'GS',
    '2-C60M': 'POAG',
    '2-C60P': 'PACG'
}

clinic['Condition_Name'] = clinic['Condition_Label'].map(condition_map)

# Create the second new column with Glaucoma classification
def classify_glaucoma(condition):
    if condition in ['POAG', 'PACG']:
        return 'Glaucoma'
    elif condition == 'GS':
        return 'Glaucoma suspects'
    else:
        return np.nan

clinic['Glaucoma_Classification'] = clinic['Condition_Name'].apply(classify_glaucoma)

clinic

,RID,RCT,RMT,RCB,RMB,Date_of_Encounter,LogMAR_VA,Visual_Acuity_Numerator,IOP,Refractive_Error,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label,Condition_Name,Glaucoma_Classification
0,2-7XT4,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-08-05 00:00:00,0.14,20.0,13.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130,2-C60M,POAG,Glaucoma
1,2-7XT6,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-08-05 00:00:00,0.22,20.0,14.0,NaN,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131,2-C60M,POAG,Glaucoma
2,2-7XT8,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-10-22 00:00:00,0.00,20.0,14.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132,2-C60M,POAG,Glaucoma
3,2-7XTA,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2020-10-22 00:00:00,0.00,20.0,13.0,NaN,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133,2-C60M,POAG,Glaucoma
4,2-7XTC,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:09.294098+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2023-04-18 00:00:00,0.18,20.0,9.0,-0.5,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847,2-C60M,POAG,Glaucoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4332,2-868W,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2023-04-19 00:00:00,0.18,20.0,19.0,NaN,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014,2-C60J,GS,Glaucoma suspects
4333,2-868Y,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-10-19 00:00:00,0.18,20.0,16.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728,2-C60J,GS,Glaucoma suspects
4334,2-8690,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-10-19 00:00:00,0.22,20.0,16.0,NaN,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729,2-C60J,GS,Glaucoma suspects
4335,2-8692,2024-04-15 23:52:33.510438+00,2024-06-11 00:17:10.417504+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2022-04-20 00:00:00,0.18,20.0,14.0,NaN,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726,2-C60J,GS,Glaucoma suspects


In [37]:
clinic['Glaucoma_Classification'].unique()

array(['Glaucoma', 'Glaucoma suspects', nan], dtype=object)

In [38]:
subject_obs_clinic_data = pd.merge(subject_obs_clinic, clinic, left_on='Clinical_Records', right_on='RID', suffixes=("_subject_obs_clinic_df", "_clinic_df"), how='left')




subject_obs_clinic_data

,RID_subject_df,RCT_subject_df,RMT_subject_df,RCB_subject_df,RMB_subject_df,Subject_ID,hba1c,glaucoma_hx,visual_acuity_right,visual_acuity_left,...,CCT,CDR,Gonioscopy,Condition_Display,Provider,Powerform_Laterality,Clinical_ID,Condition_Label,Condition_Name,Glaucoma_Classification
0,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK0,3130.0,2-C60M,POAG,Glaucoma
1,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Unspecified osteoarthritis, unspecified site_ ...",Andrew Clark,2SK2,3131.0,2-C60M,POAG,Glaucoma
2,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK0,3132.0,2-C60M,POAG,Glaucoma
3,2-7KTR,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1,NaN,NaN,NaN,NaN,...,NaN,0.60,NaN,"Primary open-angle glaucoma, bilateral, modera...",Linda Lam,2SK2,3133.0,2-C60M,POAG,Glaucoma
4,2-7KTT,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,2,NaN,NaN,NaN,NaN,...,NaN,0.70,4/4/4/4,"Primary open-angle glaucoma, bilateral, modera...",Van Nguyen,2SK0,52847.0,2-C60M,POAG,Glaucoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,Wilson's disease_ Type 2 diabetes mellitus wit...,Brian Song,2SK2,60014.0,2-C60J,GS,Glaucoma suspects
4971,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44728.0,2-C60J,GS,Glaucoma suspects
4972,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK2,44729.0,2-C60J,GS,Glaucoma suspects
4973,2-7P5M,2024-04-15 23:44:22.184165+00,2024-04-15 23:47:57.356528+00,https://auth.globus.org/3769492a-b197-4063-952...,https://auth.globus.org/3769492a-b197-4063-952...,1199,NaN,NaN,NaN,NaN,...,NaN,0.70,NaN,"Wilson's disease_ Vitreous degeneration, bilat...",Brian Song,2SK0,44726.0,2-C60J,GS,Glaucoma suspects


In [40]:
subject_obs_clinic_data['Glaucoma_Classification'].unique()

array(['Glaucoma', 'Glaucoma suspects', nan], dtype=object)

In [42]:
subject_obs_clinic_data['RID_subject_df'].nunique()

1199

In [43]:

# Assuming subject_obs_clinic_data is your final merged dataframe
unique_subject_counts = subject_obs_clinic_data.groupby('Glaucoma_Classification')['RID_subject_df'].nunique()

# Print the unique subject counts for each classification
print(unique_subject_counts)

Glaucoma_Classification
Glaucoma             708
Glaucoma suspects    445
Name: RID_subject_df, dtype: int64


In [44]:

# Assuming subject_obs_clinic_data is your final merged dataframe
unique_subject_counts = subject_obs_clinic_data.groupby('Glaucoma_Classification')['Subject_ID'].nunique()

# Print the unique subject counts for each classification
print(unique_subject_counts)

Glaucoma_Classification
Glaucoma             708
Glaucoma suspects    445
Name: Subject_ID, dtype: int64


In [ ]:
# # @title Execute Training algorithm
# from eye_ai.models.vgg19_hyper_parameter_tuning import main #import the new logistic module.
# with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
#   main()


In [ ]:
# # @title Save Execution Assets (model) and Metadata
# uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

# 